In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *
from sklearn.metrics import *

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

/Users/noharam/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


TensorFlow version: 2.9.1
Eager execution: True


## 훈련 데이터 가져오기 및 파싱

데이터 파일을 다운로드하여 이 Python 프로그램이 사용할 수 있는 구조로 해당 데이터를 전환합니다.

데이터세트 다운로드
tf.keras.utils.get_file 함수를 사용하여 훈련 데이터세트를 다운로드합니다. 이 함수는 다운로드된 파일 경로를 반환합니다.

In [2]:
# Dataset Load
train_dataset_url = (
    "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
train_dataset_fp = tf.keras.utils.get_file(
    fname=os.path.basename(train_dataset_url), origin=train_dataset_url
)
print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: /home/sysadmin/.keras/datasets/iris_training.csv


## 데이터 탐색

이 데이터세트 iris_training.csv는 텍스트 파일이며, 표로 된 데이터를 CSV(comma-separated values)로 저장합니다. head -n5 명령을 사용하여 처음 5개 항목을 확인합니다.

In [5]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


해당 데이터세트에서 다음 사항에 주목하세요.

1. 첫 번째 줄은 데이터세트 정보를 포함하는 헤더입니다.
    - 총 120개의 샘플이 있으며, 각 샘플에는 4개의 특성과 3개의 가능한 라벨 이름 중 하나가 있습니다.
2. 다음 줄은 데이터 레코드로, 한 줄당 한 개의 예가 있습니다.
    - 처음 4개의 필드는 특성으로, 예의 특징을 보여줍니다. 여기서 필드는 붓꽃의 측정값을 부동소수점으로 표시합니다.
     - 마지막 열은 라벨이며 예측하려는 값을 나타냅니다. 이 데이터세트에서는 꽃의 이름에 상응하는 정수값 0, 1, 2를 나타냅니다.

코드로 표현하면 다음과 같습니다.

In [6]:
# column order in CSV file
column_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
feature_names = column_names[:-1]
label_name = column_names[-1]
print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

class_names = ["Iris setosa", "Iris versicolor", "Iris virginica"]

Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


각 라벨은 'setosa'와 같은 문자형 이름과 관련이 있습니다. 하지만 머신러닝은 주로 숫자형 값에 의존합니다. 라벨 숫자는 다음의 이름을 대신합니다.

- 0: Iris setosa
- 1: Iris versicolor
- 2: Iris virginica

## tf.data.Dataset 생성

TensorFlow의 데이터세트 API는 모델로 데이터를 로드할 때 일반적으로 발생하는 다양한 사례를 다룹니다. 이는 데이터를 읽고 훈련에 필요한 형태로 변환하는 고수준 API입니다.

데이터세트는 CSV 형식의 텍스트 파일이므로, 적절한 형태로 데이터를 구분하기 위해 tf.data.experimental.make_csv_dataset 함수를 사용하겠습니다.

In [7]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1,
)

make_csv_dataset 함수는 (features, label) 쌍에서 tf.data.Dataset를 반환하며, 여기서 features는 {'feature_name': value} 사전에 해당합니다.

이러한 Dataset 객체는 반복 가능합니다. 다음을 통해 배치별 특성을 살펴보겠습니다.

In [8]:

features, labels = next(iter(train_dataset))
print(features)

OrderedDict([('sepal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([5. , 5.1, 5.1, 7.2, 6.4, 6.2, 5.1, 4.9, 5.2, 6.3, 6.5, 5.5, 5.2,
       6.9, 5.4, 6. , 7. , 7.4, 4.8, 6.6, 6.7, 5.1, 4.6, 7.3, 4.9, 5.4,
       6.3, 7.7, 7.6, 6.7, 7.7, 4.6], dtype=float32)>), ('sepal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([3.6, 3.8, 3.8, 3. , 2.8, 3.4, 2.5, 3.1, 2.7, 3.3, 3.2, 2.6, 3.4,
       3.1, 3.7, 2.2, 3.2, 2.8, 3.4, 2.9, 3.1, 3.8, 3.6, 2.9, 2.5, 3.9,
       2.5, 2.6, 3. , 3. , 3. , 3.4], dtype=float32)>), ('petal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1.4, 1.5, 1.9, 5.8, 5.6, 5.4, 3. , 1.5, 3.9, 4.7, 5.1, 4.4, 1.4,
       5.1, 1.5, 5. , 4.7, 6.1, 1.6, 4.6, 5.6, 1.6, 1. , 6.3, 4.5, 1.7,
       5. , 6.9, 6.6, 5.2, 6.1, 1.4], dtype=float32)>), ('petal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0.2, 0.3, 0.4, 1.6, 2.1, 2.3, 1.1, 0.1, 1.4, 1.6, 2. , 1.2, 0.2,
       2.3, 0.2, 1.5, 1.4, 1.9, 0.2, 1.3, 2.4, 0.2, 0.2, 1.8,

유사한 특성은 하나의 그룹으로 묶이거나 배치 처리된다는 점에 주목하세요. 각 예제 행의 필드는 해당하는 특성 배열에 추가됩니다. batch_size를 조정하여 이러한 특성 배열에 저장된 샘플 수를 설정하세요.

또한 배치에서 일부 특성을 플롯하여 클러스터가 생기는 것을 확인할 수 있습니다.

모델 구축 단계를 단순화하기 위해, 특성 사전을 (batch_size, num_features)의 형상을 갖는 단일 배열로 리패키징하는 함수를 생성합니다.

이 함수는 tf.stack 메서드를 사용하여 텐서의 목록에서 값을 취하고 지정된 차원에서 결합된 텐서를 생성합니다

In [9]:
def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features.values()), axis=1)
    return features, labels

그런 다음 tf.data.Dataset.map 메서드를 사용하여 각 (features,label) 쌍의 features을 훈련 데이터세트에 저장합니다.

In [10]:
train_dataset = train_dataset.map(pack_features_vector)

features, labels = next(iter(train_dataset))
print(features[:5])

tf.Tensor(
[[6.8 2.8 4.8 1.4]
 [6.9 3.2 5.7 2.3]
 [6.7 3.1 5.6 2.4]
 [7.7 3.8 6.7 2.2]
 [5.  3.4 1.5 0.2]], shape=(5, 4), dtype=float32)


## 케라스를 사용한 모델 생성

TensorFlow의 tf.keras API는 모델과 레이어를 생성하는 데 주로 사용됩니다. Keras가 모든 구성 요소 연결에 대한 복잡성을 처리해 주기 때문에 모델을 구축하고 실험하는 데 용이합니다.

tf.keras.Sequential 모델은 레이어의 선형 스택입니다. 이 생성자는 레이어 인스턴스 목록을 취하는데, 아래의 경우, 각 10개의 노드를 갖는 두 개의 tf.keras.layers.Dense 레이어 및 3개의 노드를 갖는 출력 레이어로 구성되어 레이블 예측을 보여주고 있습니다. 첫 번째 레이어의 input_shape 매개변수는 데이터세트의 특성 수에 해당하며 필수적입니다.

In [5]:
model = Sequential()
model.add(Dense(10, activation=tf.nn.relu, input_shape=(4,)))
model.add(Dense(10, activation=tf.nn.relu))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

활성화 함수는 각 레이어의 노드에서 출력 형상을 결정합니다. 이러한 비선형성이 중요한데, 활성화 함수가 없는 모델은 단일 레이어와 마찬가지이기 때문입니다. tf.keras.activations가 많이 있지만, 숨겨진 레이어에서는 주로 ReLU 함수가 사용됩니다.

## 모델 훈련하기

In [ ]:
model.fit(features, labels, epochs=200)

## 모델 유효성 평가

이제 모델은 훈련되었습니다. 모델의 성능에 대한 몇가지 통계를 얻을 수 있습니다.

평가는 모델이 얼마나 효과적으로 예측을 수행하는지 알아보는 것을 의미합니다. 붓꽃 분류에서 모델의 효과를 확인하려면 꽃받침과 꽃잎 측정치를 모델에 전달하고 모델이 붓꽃 종을 예측하도록 요청합니다. 그런 다음 모델의 예측을 실제 레이블과 비교합니다.

In [16]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url), origin=test_url)

test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name="species",
    num_epochs=1,
    shuffle=False,
)

test_dataset = test_dataset.map(pack_features_vector)
test_features, test_labels = next(iter(train_dataset))

result = model.evaluate(test_features,  test_labels, verbose=2)
print("accuracy: {:.4}".format(result[1]))

Test set accuracy: 96.667%
